# CHAPTER 5 실습: 상용화 준비

이 노트북은 『MLOps 도입 가이드』 5장 상용화 준비 내용을 기반으로 한 실습입니다.

실습 항목:
- 모델 리스크 평가
- 품질 검증
- 예측 신뢰도 기반 리스크 감지

## 1. 데이터 로딩 및 전처리

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# 타이타닉 데이터 로딩
df = sns.load_dataset("titanic").dropna(subset=["age", "embarked"])
df["sex"] = df["sex"].map({"male": 0, "female": 1})
df["embarked"] = df["embarked"].astype("category").cat.codes
features = ["pclass", "sex", "age", "sibsp", "parch", "fare", "embarked"]
X = df[features]
y = df["survived"]

## 2. 모델 리스크 평가: 클래스 불균형, 노이즈 반응

In [2]:
model = RandomForestClassifier(random_state=42)

# 클래스 분포
print("클래스 분포:", np.bincount(y))

# 교차 검증 정확도
scores = cross_val_score(model, X, y, cv=5, scoring="accuracy")
print("Cross-validation accuracy:", scores)
print("평균 정확도:", np.mean(scores))

클래스 분포: [424 288]
Cross-validation accuracy: [0.74125874 0.8041958  0.8028169  0.77464789 0.83802817]
평균 정확도: 0.7921895006402048


## 3. 품질 검증: 입력값 노이즈 추가

In [3]:
X_noisy = X.copy()
X_noisy["fare"] = X_noisy["fare"] + np.random.normal(0, 50, size=len(X))
scores_noisy = cross_val_score(model, X_noisy, y, cv=5)
print("노이즈 추가 후 정확도:", np.mean(scores_noisy))

노이즈 추가 후 정확도: 0.7908105978528515


## 4. 예측 신뢰도 기반 리스크 감지

In [4]:
model.fit(X, y)
proba = model.predict_proba(X)
confidence = np.max(proba, axis=1)
low_confidence = confidence < 0.6
print("불확실 예측 비율:", np.mean(low_confidence))

불확실 예측 비율: 0.028089887640449437


✅ 이 실습은 운영 전 테스트로 성능 저하 요인을 분석하고, 예측 불확실성을 감지하는 방법을 보여줍니다.